In [7]:
import pandas as pd

### Config ###
df = pd.read_csv('Cyclone_List_to_Collect_New.csv')
# Parse the 'Date Landfall/Closest to Land' to datetime
def parse_custom_date(date_str):
    try:
        return pd.to_datetime(date_str, errors='coerce')
    except:
        return None
df['Date Landfall/Closest to Land'] = df['Date Landfall/Closest to Land'].apply(parse_custom_date)
df = df.rename(columns={"Date Landfall/Closest to Land": "LandfallDate"})

C:\Users\mobility\AppData\Local\Temp\ipykernel_15412\2150577891.py:8: FutureWarning: Parsed string "May 10, 2015 - 5 AM EDT" included an un-recognized timezone "EDT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  return pd.to_datetime(date_str, errors='coerce')
C:\Users\mobility\AppData\Local\Temp\ipykernel_15412\2150577891.py:8: FutureWarning: Parsed string "June 16, 2015 - 4 PM CDT" included an un-recognized timezone "CDT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  return pd.to_datetime(date_str, errors='coerce')
C:\Users\mobility\AppData\Local\Temp\ipykernel_15412\2150577891.py:8: FutureWarning: Parsed string "July 12, 2015 - 8 PM PDT" included an un-recognized timezone "PDT". Dropping unrecogn

# Before 2016

In [5]:
df = df[df['NAME'].isin(['MATTHEW'])]

In [6]:
import os
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime, timedelta
from urllib.parse import urljoin
import pandas as pd

# --- Ensure Output Folders Exist ---
folders = {
    "5W": "Cyclone Graphics Archive Uncertainty Cone",
    "PROB34": "Cyclone Graphics Archive Wind",
    "public": "Cyclone Text Archive Advisory",
    "wndprb": "Cyclone Text Archive Wind"
}
for folder in folders.values():
    os.makedirs(folder, exist_ok=True)

# --- Loop Over Cyclone Records ---
for row in df.itertuples(index=False):
    year = row.SEASON
    storm_id = row.storm_id.upper()
    name = row.NAME.upper()
    landfall_time = row.LandfallDate

    for offset_hr in range(12, 121, 12):  # 12h to 120h (5 days) before
        target_time = landfall_time - timedelta(hours=offset_hr)
        suffix = f"_{offset_hr}h"

        # --- GRAPHICS DOWNLOAD ---
        for code, folder in [("5W", folders["5W"]), ("PROB34", folders["PROB34"])]:
            base_url = f"https://www.nhc.noaa.gov/archive/{year}/graphics/{storm_id.lower()}/"
            try:
                res = requests.get(base_url)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, "html.parser")

                file_prefix = f"{storm_id}{year}_{code}"
                file_entries = []

                for tr in soup.find_all("tr"):
                    cols = tr.find_all("td")
                    if len(cols) >= 3:
                        a_tag = cols[1].find("a")
                        time_text = cols[2].text.strip()
                        if a_tag and a_tag["href"].startswith(file_prefix):
                            try:
                                mod_time = datetime.strptime(time_text, "%Y-%m-%d %H:%M")
                                url = urljoin(base_url, a_tag["href"])
                                file_entries.append((url, mod_time))
                            except:
                                continue

                if file_entries:
                    url, file_time = min(file_entries, key=lambda x: abs(x[1] - target_time))
                    time_diff = abs(file_time - target_time)
                    if time_diff > timedelta(hours=12):
                        print(f"❌ No graphics for {code} {storm_id} {year} {suffix} (Too far from target time)")
                        continue
                    r = requests.get(url)
                    ext = url.split("/")[-1].split(".")[-1]
                    filename = os.path.join(folder, f"{name}_{year}{suffix}.{ext}")
                    with open(filename, "wb") as f:
                        f.write(r.content)
                    print(f"✅ Saved graphics: {filename}")
                else:
                    print(f"❌ No graphics found for {code} {storm_id} {year} {suffix}")
            except Exception as e:
                print(f"❌ Error downloading graphics {code} {storm_id} {year} {suffix}: {e}")

        # --- TEXT DOWNLOAD ---
        def download_text(product_type, save_folder):
            archive_url = f"https://www.nhc.noaa.gov/archive/{year}/{storm_id.lower()}/"
            try:
                res = requests.get(archive_url)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, "html.parser")

                comments = soup.find_all(string=lambda text: isinstance(text, Comment))
                file_entries = []
                for c in comments:
                    if c.strip().startswith(str(year)):
                        try:
                            time = datetime.strptime(c.strip(), "%Y%m%d %H%M")
                            a_tag = c.find_next_sibling("a")
                            if a_tag and product_type in a_tag["href"]:
                                url = urljoin(archive_url, a_tag["href"])
                                file_entries.append((url, time))
                        except:
                            continue

                if file_entries:
                    url, file_time = min(file_entries, key=lambda x: abs(x[1] - target_time))
                    time_diff = abs(file_time - target_time)
                    if time_diff > timedelta(hours=12):
                        print(f"❌ No text advisory for {product_type} {storm_id} {year} {suffix} (Too far from target time)")
                        return

                    page = requests.get(url)
                    soup_detail = BeautifulSoup(page.text, "html.parser")
                    pre_tag = soup_detail.select_one("div.textbackground div.textproduct pre")
                    if pre_tag:
                        filename = os.path.join(save_folder, f"{name}_{year}{suffix}.txt")
                        with open(filename, "w", encoding="utf-8") as f:
                            f.write(pre_tag.get_text())
                        print(f"✅ Saved text: {filename}")
                    else:
                        print(f"❌ No <pre> for {product_type} {storm_id} {year} {suffix}")
                else:
                    print(f"❌ No text advisory for {product_type} {storm_id} {year} {suffix}")
            except Exception as e:
                print(f"❌ Error downloading text {product_type} {storm_id} {year} {suffix}: {e}")

        download_text("public", folders["public"])
        download_text("wndprb", folders["wndprb"])

✅ Saved graphics: Cyclone Graphics Archive Uncertainty Cone\MATTHEW_2016_12h.GIF
✅ Saved graphics: Cyclone Graphics Archive Wind\MATTHEW_2016_12h.GIF
✅ Saved text: Cyclone Text Archive Advisory\MATTHEW_2016_12h.txt
✅ Saved text: Cyclone Text Archive Wind\MATTHEW_2016_12h.txt
✅ Saved graphics: Cyclone Graphics Archive Uncertainty Cone\MATTHEW_2016_24h.GIF
✅ Saved graphics: Cyclone Graphics Archive Wind\MATTHEW_2016_24h.GIF
✅ Saved text: Cyclone Text Archive Advisory\MATTHEW_2016_24h.txt
✅ Saved text: Cyclone Text Archive Wind\MATTHEW_2016_24h.txt
✅ Saved graphics: Cyclone Graphics Archive Uncertainty Cone\MATTHEW_2016_36h.GIF
✅ Saved graphics: Cyclone Graphics Archive Wind\MATTHEW_2016_36h.GIF
✅ Saved text: Cyclone Text Archive Advisory\MATTHEW_2016_36h.txt
✅ Saved text: Cyclone Text Archive Wind\MATTHEW_2016_36h.txt
✅ Saved graphics: Cyclone Graphics Archive Uncertainty Cone\MATTHEW_2016_48h.GIF
✅ Saved graphics: Cyclone Graphics Archive Wind\MATTHEW_2016_48h.GIF
✅ Saved text: Cyclone 

# After 2017

In [8]:
df = df[df['SEASON']>2016]

In [10]:
import os
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime, timedelta
from urllib.parse import urljoin
import pandas as pd

# --- Ensure Output Folders Exist ---
folders = {
    "5day": "Cyclone Graphics Archive Uncertainty Cone",
    "wind_probs_34": "Cyclone Graphics Archive Wind",
    "public": "Cyclone Text Archive Advisory",
    "wndprb": "Cyclone Text Archive Wind"
}
for folder in folders.values():
    os.makedirs(folder, exist_ok=True)

# --- Loop Over Cyclone Records ---
for row in df.itertuples(index=False):
    year = row.SEASON
    storm_id = row.storm_id[:8].upper()
    storm_id1 = row.storm_id[:8].upper()
    storm_id2 = row.storm_id[:4].lower()
    name = row.NAME.upper()
    landfall_time = row.LandfallDate

    for offset_hr in range(12, 121, 12):  # 12h to 120h (5 days) before
        target_time = landfall_time - timedelta(hours=offset_hr)
        suffix = f"_{offset_hr}h"

        # --- GRAPHICS DOWNLOAD ---
        for code, folder in [("5day", folders["5day"]), ("wind_probs_34", folders["wind_probs_34"])]:
            base_url = f"https://www.nhc.noaa.gov/archive/{year}/graphics/{storm_id}/"
            try:
                res = requests.get(base_url)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, "html.parser")
                
                file_prefix = f"{storm_id1}_{code}"
                file_entries = []

                for tr in soup.find_all("tr"):
                    cols = tr.find_all("td")
                    if len(cols) >= 3:
                        a_tag = cols[1].find("a")
                        time_text = cols[2].text.strip()
                        if a_tag and a_tag["href"].startswith(file_prefix):
                            try:
                                mod_time = datetime.strptime(time_text, "%Y-%m-%d %H:%M")
                                url = urljoin(base_url, a_tag["href"])
                                file_entries.append((url, mod_time))
                            except:
                                continue

                if file_entries:
                    url, file_time = min(file_entries, key=lambda x: abs(x[1] - target_time))
                    time_diff = abs(file_time - target_time)
                    if time_diff > timedelta(hours=12):
                        print(f"❌ No graphics for {code} {storm_id} {year} {suffix} (Too far from target time)")
                        continue
                    r = requests.get(url)
                    ext = url.split("/")[-1].split(".")[-1]
                    filename = os.path.join(folder, f"{name}_{year}{suffix}.{ext}")
                    with open(filename, "wb") as f:
                        f.write(r.content)
                    print(f"✅ Saved graphics: {filename}")
                else:
                    print(f"❌ No graphics found for {code} {storm_id} {year} {suffix}")
            except Exception as e:
                print(f"❌ Error downloading graphics {code} {storm_id} {year} {suffix}: {e}")

        # --- TEXT DOWNLOAD ---
        def download_text(product_type, save_folder):
            archive_url = f"https://www.nhc.noaa.gov/archive/{year}/{storm_id2}/"
            try:
                res = requests.get(archive_url)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, "html.parser")

                comments = soup.find_all(string=lambda text: isinstance(text, Comment))
                file_entries = []
                for c in comments:
                    if c.strip().startswith(str(year)):
                        try:
                            time = datetime.strptime(c.strip(), "%Y%m%d %H%M")
                            a_tag = c.find_next_sibling("a")
                            if a_tag and product_type in a_tag["href"]:
                                url = urljoin(archive_url, a_tag["href"])
                                file_entries.append((url, time))
                        except:
                            continue

                if file_entries:
                    url, file_time = min(file_entries, key=lambda x: abs(x[1] - target_time))
                    time_diff = abs(file_time - target_time)
                    if time_diff > timedelta(hours=12):
                        print(f"❌ No text advisory for {product_type} {storm_id} {year} {suffix} (Too far from target time)")
                        return

                    page = requests.get(url)
                    soup_detail = BeautifulSoup(page.text, "html.parser")
                    pre_tag = soup_detail.select_one("div.textbackground div.textproduct pre")
                    if pre_tag:
                        filename = os.path.join(save_folder, f"{name}_{year}{suffix}.txt")
                        with open(filename, "w", encoding="utf-8") as f:
                            f.write(pre_tag.get_text())
                        print(f"✅ Saved text: {filename}")
                    else:
                        print(f"❌ No <pre> for {product_type} {storm_id} {year} {suffix}")
                else:
                    print(f"❌ No text advisory for {product_type} {storm_id} {year} {suffix}")
            except Exception as e:
                print(f"❌ Error downloading text {product_type} {storm_id} {year} {suffix}: {e}")

        download_text("public", folders["public"])
        download_text("wndprb", folders["wndprb"])

✅ Saved graphics: Cyclone Graphics Archive Uncertainty Cone\IRMA_2017_12h.png
✅ Saved graphics: Cyclone Graphics Archive Wind\IRMA_2017_12h.png
✅ Saved text: Cyclone Text Archive Advisory\IRMA_2017_12h.txt
✅ Saved text: Cyclone Text Archive Wind\IRMA_2017_12h.txt
✅ Saved graphics: Cyclone Graphics Archive Uncertainty Cone\IRMA_2017_24h.png
✅ Saved graphics: Cyclone Graphics Archive Wind\IRMA_2017_24h.png
✅ Saved text: Cyclone Text Archive Advisory\IRMA_2017_24h.txt
✅ Saved text: Cyclone Text Archive Wind\IRMA_2017_24h.txt
✅ Saved graphics: Cyclone Graphics Archive Uncertainty Cone\IRMA_2017_36h.png
✅ Saved graphics: Cyclone Graphics Archive Wind\IRMA_2017_36h.png
✅ Saved text: Cyclone Text Archive Advisory\IRMA_2017_36h.txt
✅ Saved text: Cyclone Text Archive Wind\IRMA_2017_36h.txt
✅ Saved graphics: Cyclone Graphics Archive Uncertainty Cone\IRMA_2017_48h.png
✅ Saved graphics: Cyclone Graphics Archive Wind\IRMA_2017_48h.png
✅ Saved text: Cyclone Text Archive Advisory\IRMA_2017_48h.txt
✅ 

In [49]:
df[df['SEASON']==2022]

,SID,START_DATE,NAME,storm_id,SEASON,LandfallDate,Wind_Speed_1_Day,Wind_Speed_2_Day,Image_1_Day,Image_2_Day,Wind_Speed_Landfall/ Closest_To_Land,Second_Landfall,Third_Landfall
71,2022154N21273,6/2/2022,ALEX,AL012022/,2022,2022-06-04 11:00:00,"June 3, 2022 - 9:00 UTC",NaN,"June 3, 2022 - 10 AM CDT",NaN,NaN,NaN,NaN
72,2022183N32280,7/1/2022,COLIN,AL032022/,2022,2022-07-02 05:00:00,NaN,NaN,NaN,NaN,"July 2, 2022 - 9:00 UTC",NaN,NaN
73,2022246N18301,9/2/2022,EARL,AL062022/,2022,2022-09-04 05:00:00,"September 3, 2022 - 3:00 UTC",NaN,"September 3, 2022 - 5 AM AST",NaN,NaN,NaN,NaN
74,2022248N14259,9/4/2022,KAY,EP122022/,2022,2022-09-08 15:00:00,"September 7, 2022 - 15:00 UTC","September 6, 2022 - 15:00 UTC","September 7, 2022 - 3 PM MDT","September 6, 2022 - 3 PM MDT",NaN,NaN,NaN
75,2022257N16312,9/14/2022,FIONA,AL072022/,2022,2022-09-19 02:00:00,"September 18, 2022 - 3:00 UTC","September 17, 2022 - 3:00 UTC","September 18, 2022 - 2 AM AST","September 17, 2022 - 2 AM AST",NaN,"September 24, 2022 - 5 AM AST",NaN
76,2022266N12294,9/22/2022,IAN,AL092022/,2022,2022-09-27 08:00:00,"September 26, 2022 - 9:00 UTC","September 25, 2022 - 9:00 UTC","September 26, 2022 - 8 AM EDT","September 25, 2022 - 8 AM EDT",NaN,"September 28, 2022 - 5 PM EDT",NaN
77,2022293N15259,10/20/2022,ROSLYN,EP192022/,2022,2022-10-23 06:00:00,"October 22, 2022 - 9:00 UTC","October 21, 2022 - 9:00 UTC","October 22, 2022 - 6 AM MDT","October 21, 2022 - 7 AM CDT",NaN,NaN,NaN
78,2022311N21293,11/6/2022,NICOLE,AL172022/,2022,2022-11-10 04:00:00,"November 9, 2022 - 3:00 UTC","November 8, 2022 - 3:00 UTC","November 9, 2022 - 4 AM EST","November 8, 2022 - 4 AM EST",NaN,NaN,NaN


In [55]:
# Every one day

import os
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime, timedelta
from urllib.parse import urljoin
import pandas as pd

# --- Ensure Output Folders Exist ---
folders = {
    "5W": "Cyclone Graphics Archive Uncertainty Cone",
    "PROB34": "Cyclone Graphics Archive Wind",
    "public": "Cyclone Text Archive Advisory",
    "wndprb": "Cyclone Text Archive Wind"
}
for folder in folders.values():
    os.makedirs(folder, exist_ok=True)

# --- Loop Over Cyclone Records ---
for row in df.itertuples(index=False):
    year = row.SEASON
    storm_id = row.storm_id.upper()
    name = row.NAME.upper()
    landfall_time = row.LandfallDate

    for day_offset in range(1, 6):  # 1 to 5 days before
        target_time = landfall_time - timedelta(days=day_offset)
        suffix = f"_{day_offset}"

        # --- GRAPHICS DOWNLOAD ---
        for code, folder in [("5W", folders["5W"]), ("PROB34", folders["PROB34"])]:
            base_url = f"https://www.nhc.noaa.gov/archive/{year}/graphics/{storm_id.lower()}/"
            try:
                res = requests.get(base_url)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, "html.parser")

                file_prefix = f"{storm_id}{year}_{code}"
                file_entries = []

                for tr in soup.find_all("tr"):
                    cols = tr.find_all("td")
                    if len(cols) >= 3:
                        a_tag = cols[1].find("a")
                        time_text = cols[2].text.strip()
                        if a_tag and a_tag["href"].startswith(file_prefix):
                            try:
                                mod_time = datetime.strptime(time_text, "%Y-%m-%d %H:%M")
                                url = urljoin(base_url, a_tag["href"])
                                file_entries.append((url, mod_time))
                            except:
                                continue

                if file_entries:
                    url, file_time = min(file_entries, key=lambda x: abs(x[1] - target_time))
                    time_diff = abs(file_time - target_time)
                    if time_diff > timedelta(hours=12):
                        print(f"❌ No graphics for {code} {storm_id} {year} Day -{day_offset} (Too far from target time)")
                        continue
                    r = requests.get(url)
                    ext = url.split("/")[-1].split(".")[-1]
                    filename = os.path.join(folder, f"{name}{suffix}.{ext}")
                    with open(filename, "wb") as f:
                        f.write(r.content)
                    print(f"✅ Saved graphics: {filename}")
                else:
                    print(f"❌ No graphics found for {code} {storm_id} {year} Day -{day_offset}")
            except Exception as e:
                print(f"❌ Error downloading graphics {code} {storm_id} {year} Day -{day_offset}: {e}")

        # --- TEXT DOWNLOAD ---
        def download_text(product_type, save_folder):
            archive_url = f"https://www.nhc.noaa.gov/archive/{year}/{storm_id.lower()}/"
            try:
                res = requests.get(archive_url)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, "html.parser")

                comments = soup.find_all(string=lambda text: isinstance(text, Comment))
                file_entries = []
                for c in comments:
                    if c.strip().startswith(str(year)):
                        try:
                            time = datetime.strptime(c.strip(), "%Y%m%d %H%M")
                            a_tag = c.find_next_sibling("a")
                            if a_tag and product_type in a_tag["href"]:
                                url = urljoin(archive_url, a_tag["href"])
                                file_entries.append((url, time))
                        except:
                            continue

                if file_entries:
                    url, file_time = min(file_entries, key=lambda x: abs(x[1] - target_time))
                    time_diff = abs(file_time - target_time)
                    if time_diff > timedelta(hours=12):
                        print(f"❌ No text advisory for {product_type} {storm_id} {year} Day -{day_offset} (Too far from target time)")
                        return

                    page = requests.get(url)
                    soup_detail = BeautifulSoup(page.text, "html.parser")
                    pre_tag = soup_detail.select_one("div.textbackground div.textproduct pre")
                    if pre_tag:
                        filename = os.path.join(save_folder, f"{name}{suffix}.txt")
                        with open(filename, "w", encoding="utf-8") as f:
                            f.write(pre_tag.get_text())
                        print(f"✅ Saved text: {filename}")
                    else:
                        print(f"❌ No <pre> for {product_type} {storm_id} {year} Day -{day_offset}")
                else:
                    print(f"❌ No text advisory for {product_type} {storm_id} {year} Day -{day_offset}")
            except Exception as e:
                print(f"❌ Error downloading text {product_type} {storm_id} {year} Day -{day_offset}: {e}")

        download_text("public", folders["public"])
        download_text("wndprb", folders["wndprb"])

✅ Saved graphics: Cyclone Graphics Archive Uncertainty Cone\ANA_1.GIF
✅ Saved graphics: Cyclone Graphics Archive Wind\ANA_1.GIF
✅ Saved text: Cyclone Text Archive Advisory\ANA_1.txt
✅ Saved text: Cyclone Text Archive Wind\ANA_1.txt
✅ Saved graphics: Cyclone Graphics Archive Uncertainty Cone\ANA_2.GIF
✅ Saved graphics: Cyclone Graphics Archive Wind\ANA_2.GIF
✅ Saved text: Cyclone Text Archive Advisory\ANA_2.txt
✅ Saved text: Cyclone Text Archive Wind\ANA_2.txt
❌ No graphics for 5W AL01 2015 Day -3 (Too far from target time)
❌ No graphics for PROB34 AL01 2015 Day -3 (Too far from target time)
❌ No text advisory for public AL01 2015 Day -3 (Too far from target time)
❌ No text advisory for wndprb AL01 2015 Day -3 (Too far from target time)
❌ No graphics for 5W AL01 2015 Day -4 (Too far from target time)
❌ No graphics for PROB34 AL01 2015 Day -4 (Too far from target time)
❌ No text advisory for public AL01 2015 Day -4 (Too far from target time)
❌ No text advisory for wndprb AL01 2015 Day -4

In [31]:
import os
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime
from urllib.parse import urljoin
import pandas as pd

# --- YOUR STORM DATAFRAME ---
df = pd.DataFrame([
    {
        'SEASON': 2015,
        'storm_id': 'AL01',
        'NAME': 'ANA',
        'Date Landfall/Closest to Land': datetime.strptime("2015-05-10 17:55", "%Y-%m-%d %H:%M")
    }
])

# --- OUTPUT FOLDERS ---
folders = {
    "5W": "Cyclone Graphics Archive Uncertainty Cone",
    "PROB34": "Cyclone Graphics Archive Wind",
    "public": "Cyclone Text Archive Advisory",
    "wndprb": "Cyclone Text Archive Wind"
}
for folder in folders.values():
    os.makedirs(folder, exist_ok=True)

# --- MAIN LOOP ---
for row in df.itertuples(index=False):
    year = row.SEASON
    storm_id = row.storm_id.upper()
    name = row.NAME.upper()
    target_time = row._3  # Date Landfall/Closest to Land

    # --- GRAPHICS DOWNLOAD (5W and PROB34) ---
    for code, folder in [("5W", folders["5W"]), ("PROB34", folders["PROB34"])]:
        base_url = f"https://www.nhc.noaa.gov/archive/{year}/graphics/{storm_id.lower()}/"
        try:
            res = requests.get(base_url)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, "html.parser")

            file_prefix = f"{storm_id}{year}_{code}"
            file_entries = []

            for tr in soup.find_all("tr"):
                cols = tr.find_all("td")
                if len(cols) >= 3:
                    a_tag = cols[1].find("a")
                    time_text = cols[2].text.strip()
                    if a_tag and a_tag["href"].startswith(file_prefix):
                        try:
                            mod_time = datetime.strptime(time_text, "%Y-%m-%d %H:%M")
                            url = urljoin(base_url, a_tag["href"])
                            file_entries.append((url, mod_time))
                        except:
                            continue

            if file_entries:
                url, _ = min(file_entries, key=lambda x: abs(x[1] - target_time))
                r = requests.get(url)
                ext = url.split("/")[-1].split(".")[-1]
                suffix = "_wind" if "PROB34" in code else ""
                filename = os.path.join(folder, f"{name}{suffix}.{ext}")
                with open(filename, "wb") as f:
                    f.write(r.content)
                print(f"✅ Saved graphics: {filename}")
            else:
                print(f"❌ No graphics found for {code} in {storm_id} {year}")
        except Exception as e:
            print(f"❌ Error downloading graphics for {code} {storm_id} {year}: {e}")

    # --- TEXT DOWNLOAD (public and wndprb) ---
    def download_text(product_type, save_folder, suffix):
        archive_url = f"https://www.nhc.noaa.gov/archive/{year}/{storm_id.lower()}/"
        try:
            res = requests.get(archive_url)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, "html.parser")

            comments = soup.find_all(string=lambda text: isinstance(text, Comment))
            file_entries = []
            for c in comments:
                if c.strip().startswith(str(year)):
                    try:
                        time = datetime.strptime(c.strip(), "%Y%m%d %H%M")
                        a_tag = c.find_next_sibling("a")
                        if a_tag and product_type in a_tag["href"]:
                            url = urljoin(archive_url, a_tag["href"])
                            file_entries.append((url, time))
                    except:
                        continue

            if file_entries:
                url, _ = min(file_entries, key=lambda x: abs(x[1] - target_time))
                page = requests.get(url)
                soup_detail = BeautifulSoup(page.text, "html.parser")
                pre_tag = soup_detail.select_one("div.textbackground div.textproduct pre")
                if pre_tag:
                    filename = os.path.join(save_folder, f"{name}{suffix}.txt")
                    with open(filename, "w", encoding="utf-8") as f:
                        f.write(pre_tag.get_text())
                    print(f"✅ Saved text: {filename}")
                else:
                    print(f"❌ No <pre> found for {product_type} in {storm_id} {year}")
            else:
                print(f"❌ No text advisory found for {product_type} in {storm_id} {year}")
        except Exception as e:
            print(f"❌ Error downloading text for {product_type} {storm_id} {year}: {e}")

    download_text("public", folders["public"], "_advisory")
    download_text("wndprb", folders["wndprb"], "_wind")

✅ Saved graphics: Cyclone Graphics Archive Uncertainty Cone\ANA.GIF
✅ Saved graphics: Cyclone Graphics Archive Wind\ANA_wind.GIF
✅ Saved text: Cyclone Text Archive Advisory\ANA_advisory.txt
✅ Saved text: Cyclone Text Archive Wind\ANA_wind.txt


In [37]:
# Re-run the script after execution environment reset

import os
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime
from urllib.parse import urljoin
import pandas as pd

# Sample DataFrame input (replace this with actual data)
df = pd.DataFrame([
    {
        'SEASON': 2015,
        'storm_id': 'AL01',
        'NAME': 'ANA',
        'Date Landfall/Closest to Land': datetime.strptime("2015-05-10 17:55", "%Y-%m-%d %H:%M")
    }
])

# Ensure output directories exist
os.makedirs("Cyclone Graphics Archive Uncertainty Cone", exist_ok=True)
os.makedirs("Cyclone Graphics Archive Wind", exist_ok=True)
os.makedirs("Cyclone Text Archive Advisory", exist_ok=True)
os.makedirs("Cyclone Text Archive Wind", exist_ok=True)

for row in df.itertuples(index=False):
    year = row.SEASON
    storm_id = row.storm_id.upper()
    name = row.NAME
    target_time = row._3  # Date Landfall/Closest to Land

    # --- GRAPHICS DOWNLOAD ---
    product_codes = [("5W", "Cyclone Graphics Archive Uncertainty Cone"),
                     ("PROB34", "Cyclone Graphics Archive Wind")]
    base_url = f"https://www.nhc.noaa.gov/archive/{year}/graphics/{storm_id.lower()}/"

    try:
        res = requests.get(base_url)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")

        for product_code, folder in product_codes:
            file_prefix = f"{storm_id}{year}_{product_code}"
            file_entries = []

            for tr in soup.find_all("tr"):
                cols = tr.find_all("td")
                if len(cols) >= 3:
                    a_tag = cols[1].find("a")
                    mod_time_text = cols[2].text.strip()
                    if a_tag and a_tag["href"].startswith(file_prefix):
                        file_url = urljoin(base_url, a_tag["href"])
                        try:
                            mod_time = datetime.strptime(mod_time_text, "%Y-%m-%d %H:%M")
                            file_entries.append((file_url, mod_time))
                        except ValueError:
                            continue

            if file_entries:
                closest_file = min(file_entries, key=lambda x: abs(x[1] - target_time))
                r = requests.get(closest_file[0])
                filename = os.path.join(folder, closest_file[0].split("/")[-1])
                with open(filename, "wb") as f:
                    f.write(r.content)
    except Exception as e:
        print(f"Error downloading graphics for {storm_id} {year}: {e}")

    # --- TEXT ADVISORY DOWNLOAD ---
    def download_text_product(product_type, save_folder):
        archive_url = f"https://www.nhc.noaa.gov/archive/{year}/{storm_id.lower()}/"
        try:
            res = requests.get(archive_url)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, "html.parser")
            comments = soup.find_all(string=lambda text: isinstance(text, Comment))
            file_entries = []

            for comment in comments:
                comment_text = comment.strip()
                if comment_text.startswith(str(year)):
                    try:
                        dt = datetime.strptime(comment_text, "%Y%m%d %H%M")
                    except ValueError:
                        continue

                    a_tag = comment.find_next_sibling("a")
                    if a_tag and a_tag.get("href"):
                        href = a_tag["href"]
                        if product_type in href:
                            full_url = urljoin(archive_url, href)
                            file_entries.append((full_url, dt))

            if file_entries:
                closest_file = min(file_entries, key=lambda x: abs(x[1] - target_time))
                page = requests.get(closest_file[0])
                soup_detail = BeautifulSoup(page.text, "html.parser")
                pre_tag = soup_detail.select_one("div.textbackground div.textproduct pre")
                if pre_tag:
                    text = pre_tag.get_text()
                    filename = closest_file[0].split("/")[-1].split("?")[0].replace(".shtml", ".txt")
                    with open(os.path.join(save_folder, filename), "w", encoding="utf-8") as f:
                        f.write(text)
        except Exception as e:
            print(f"Error downloading {product_type} text for {storm_id} {year}: {e}")

    download_text_product("public", "Cyclone Text Archive Advisory")
    download_text_product("wndprb", "Cyclone Text Archive Wind")

In [31]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime

# --- CONFIG ---
product_codes = ["5W", "PROB34"]  # Multiple product codes
target_time = datetime.strptime("2015-05-10 03:00", "%Y-%m-%d %H:%M")
base_url = f"https://www.nhc.noaa.gov/archive/{year}/graphics/{storm_id.lower()}/"

# --- SCRAPE & DOWNLOAD LOOP ---
res = requests.get(base_url)
res.raise_for_status()
soup = BeautifulSoup(res.text, "html.parser")

for product_code in product_codes:
    file_prefix = f"{storm_id}{year}_{product_code}"
    file_entries = []

    for row in soup.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) >= 3:
            a_tag = cols[1].find("a")
            mod_time_text = cols[2].text.strip()
            if a_tag and a_tag["href"].startswith(file_prefix):
                file_url = urljoin(base_url, a_tag["href"])
                try:
                    mod_time = datetime.strptime(mod_time_text, "%Y-%m-%d %H:%M")
                    file_entries.append((file_url, mod_time))
                except ValueError:
                    continue  # skip malformed time entries

    if not file_entries:
        print(f"❌ No matching files starting with {file_prefix} found.")
    else:
        closest_file = min(file_entries, key=lambda x: abs(x[1] - target_time))
        print(f"📄 Closest file for {product_code}: {closest_file[0]} (Modified: {closest_file[1]})")

        # --- DOWNLOAD ---
        r = requests.get(closest_file[0])
        filename = closest_file[0].split("/")[-1]
        with open(filename, "wb") as f:
            f.write(r.content)
        print(f"✅ Downloaded to: {filename}")

📄 Closest file for 5W: https://www.nhc.noaa.gov/archive/2015/graphics/al01/AL012015_5W_009_0.GIF (Modified: 2015-05-10 02:45:00)
✅ Downloaded to: AL012015_5W_009_0.GIF
📄 Closest file for PROB34: https://www.nhc.noaa.gov/archive/2015/graphics/al01/AL012015_PROB34_009_F120.GIF (Modified: 2015-05-10 03:03:00)
✅ Downloaded to: AL012015_PROB34_009_F120.GIF


In [32]:
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime
from urllib.parse import urljoin

# --- CONFIGURATION ---
storm_id = "al01"
year = 2015
product_type = "public"
target_time = datetime.strptime("2015-05-10 17:55", "%Y-%m-%d %H:%M")
archive_url = f"https://www.nhc.noaa.gov/archive/{year}/{storm_id}/"

# --- FETCH ARCHIVE PAGE ---
res = requests.get(archive_url)
res.raise_for_status()
soup = BeautifulSoup(res.text, "html.parser")

# --- FIND FILES WITH TIME IN COMMENT ---
file_entries = []
comments = soup.find_all(string=lambda text: isinstance(text, Comment))

for comment in comments:
    comment_text = comment.strip()
    if comment_text.startswith("2015"):
        try:
            dt = datetime.strptime(comment_text, "%Y%m%d %H%M")
        except ValueError:
            continue

        a_tag = comment.find_next_sibling("a")
        if a_tag and a_tag.get("href"):
            href = a_tag["href"]
            if product_type in href:
                full_url = urljoin(archive_url, href)
                file_entries.append((full_url, dt))

# --- FIND CLOSEST FILE ---
if not file_entries:
    print("❌ No matching advisories found.")
else:
    closest_file = min(file_entries, key=lambda x: abs(x[1] - target_time))
    url_to_download = closest_file[0]
    print(f"📄 Closest advisory: {url_to_download} (Issued: {closest_file[1]})")

    # --- DOWNLOAD AND EXTRACT <pre> ---
    page = requests.get(url_to_download)
    soup_detail = BeautifulSoup(page.text, "html.parser")
    pre_tag = soup_detail.select_one("div.textbackground div.textproduct pre")

    if pre_tag:
        text = pre_tag.get_text()
        filename = url_to_download.split("/")[-1].split("?")[0].replace(".shtml", ".txt")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(text)
        print(f"✅ Advisory text saved to: {filename}")
    else:
        print("❌ <pre> tag not found in the advisory page.")

📄 Closest advisory: https://www.nhc.noaa.gov/archive/2015/al01/al012015.public.012.shtml (Issued: 2015-05-10 17:00:00)
✅ Advisory text saved to: al012015.public.012.txt


In [33]:
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime
from urllib.parse import urljoin

# --- CONFIGURATION ---
storm_id = "al01"
year = 2015
product_type = "wndprb"  # wind probability product
target_time = datetime.strptime("2015-05-10 17:55", "%Y-%m-%d %H:%M")
archive_url = f"https://www.nhc.noaa.gov/archive/{year}/{storm_id}/"

# --- FETCH ADVISORY ARCHIVE PAGE ---
res = requests.get(archive_url)
res.raise_for_status()
soup = BeautifulSoup(res.text, "html.parser")

# --- PARSE COMMENT TIMESTAMPS AND FIND LINKS ---
file_entries = []
comments = soup.find_all(string=lambda text: isinstance(text, Comment))

for comment in comments:
    comment_text = comment.strip()
    if comment_text.startswith("2015"):
        try:
            dt = datetime.strptime(comment_text, "%Y%m%d %H%M")
        except ValueError:
            continue

        a_tag = comment.find_next_sibling("a")
        if a_tag and a_tag.get("href"):
            href = a_tag["href"]
            if product_type in href:
                full_url = urljoin(archive_url, href)
                file_entries.append((full_url, dt))

# --- FIND CLOSEST FILE TO TARGET TIME ---
if not file_entries:
    print(f"❌ No matching {product_type} advisories found.")
else:
    closest_file = min(file_entries, key=lambda x: abs(x[1] - target_time))
    url_to_download = closest_file[0]
    print(f"📄 Closest wind probability advisory: {url_to_download} (Issued: {closest_file[1]})")

    # --- DOWNLOAD AND EXTRACT TEXT ---
    page = requests.get(url_to_download)
    soup_detail = BeautifulSoup(page.text, "html.parser")
    pre_tag = soup_detail.select_one("div.textbackground div.textproduct pre")

    if pre_tag:
        text = pre_tag.get_text()
        filename = url_to_download.split("/")[-1].split("?")[0].replace(".shtml", ".txt")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(text)
        print(f"✅ Wind probability advisory text saved to: {filename}")
    else:
        print("❌ <pre> tag not found in the wind probability advisory page.")

📄 Closest wind probability advisory: https://www.nhc.noaa.gov/archive/2015/al01/al012015.wndprb.011.shtml (Issued: 2015-05-10 15:00:00)
✅ Wind probability advisory text saved to: al012015.wndprb.011.txt


In [36]:
# Re-run the script after execution environment reset

import os
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime
from urllib.parse import urljoin
import pandas as pd

# Sample DataFrame input (replace this with actual data)
df = pd.DataFrame([
    {
        'SEASON': 2015,
        'storm_id': 'AL01',
        'NAME': 'ANA',
        'Date Landfall/Closest to Land': datetime.strptime("2015-05-10 17:55", "%Y-%m-%d %H:%M")
    }
])

# Ensure output directories exist
os.makedirs("Cyclone Graphics Archive Uncertainty Cone", exist_ok=True)
os.makedirs("Cyclone Graphics Archive Wind", exist_ok=True)
os.makedirs("Cyclone Text Archive Advisory", exist_ok=True)
os.makedirs("Cyclone Text Archive Wind", exist_ok=True)

for row in df.itertuples(index=False):
    year = row.SEASON
    storm_id = row.storm_id.upper()
    name = row.NAME
    target_time = row._3  # Date Landfall/Closest to Land

    # --- GRAPHICS DOWNLOAD ---
    product_codes = [("5W", "Cyclone Graphics Archive Uncertainty Cone"),
                     ("PROB34", "Cyclone Graphics Archive Wind")]
    base_url = f"https://www.nhc.noaa.gov/archive/{year}/graphics/{storm_id.lower()}/"

    try:
        res = requests.get(base_url)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")

        for product_code, folder in product_codes:
            file_prefix = f"{storm_id}{year}_{product_code}"
            file_entries = []

            for tr in soup.find_all("tr"):
                cols = tr.find_all("td")
                if len(cols) >= 3:
                    a_tag = cols[1].find("a")
                    mod_time_text = cols[2].text.strip()
                    if a_tag and a_tag["href"].startswith(file_prefix):
                        file_url = urljoin(base_url, a_tag["href"])
                        try:
                            mod_time = datetime.strptime(mod_time_text, "%Y-%m-%d %H:%M")
                            file_entries.append((file_url, mod_time))
                        except ValueError:
                            continue

            if file_entries:
                closest_file = min(file_entries, key=lambda x: abs(x[1] - target_time))
                r = requests.get(closest_file[0])
                filename = os.path.join(folder, closest_file[0].split("/")[-1])
                with open(filename, "wb") as f:
                    f.write(r.content)
    except Exception as e:
        print(f"Error downloading graphics for {storm_id} {year}: {e}")

    # --- TEXT ADVISORY DOWNLOAD ---
    def download_text_product(product_type, save_folder):
        archive_url = f"https://www.nhc.noaa.gov/archive/{year}/{storm_id.lower()}/"
        try:
            res = requests.get(archive_url)
            res.raise_for_status()
            soup = BeautifulSoup(res.text, "html.parser")
            comments = soup.find_all(string=lambda text: isinstance(text, Comment))
            file_entries = []

            for comment in comments:
                comment_text = comment.strip()
                if comment_text.startswith(str(year)):
                    try:
                        dt = datetime.strptime(comment_text, "%Y%m%d %H%M")
                    except ValueError:
                        continue

                    a_tag = comment.find_next_sibling("a")
                    if a_tag and a_tag.get("href"):
                        href = a_tag["href"]
                        if product_type in href:
                            full_url = urljoin(archive_url, href)
                            file_entries.append((full_url, dt))

            if file_entries:
                closest_file = min(file_entries, key=lambda x: abs(x[1] - target_time))
                page = requests.get(closest_file[0])
                soup_detail = BeautifulSoup(page.text, "html.parser")
                pre_tag = soup_detail.select_one("div.textbackground div.textproduct pre")
                if pre_tag:
                    text = pre_tag.get_text()
                    filename = closest_file[0].split("/")[-1].split("?")[0].replace(".shtml", ".txt")
                    with open(os.path.join(save_folder, filename), "w", encoding="utf-8") as f:
                        f.write(text)
        except Exception as e:
            print(f"Error downloading {product_type} text for {storm_id} {year}: {e}")

    download_text_product("public", "Cyclone Text Archive Advisory")
    download_text_product("wndprb", "Cyclone Text Archive Wind")